# Lets try to see if we really add 0dB Noise

In [5]:
# imports
from sample import calculation,helpers
import librosa as lr

In [7]:
# load Data
noise_path = helpers.NOISE_PROCESSED+"aircon_2_0.wav"
audio_path = helpers.DATA_PROCESSED+"usage_0_5.wav"
sr = helpers.SAMPLE_RATE
noise,_ = lr.load(noise_path,sr=sr)
audio,_ = lr.load(audio_path,sr=sr)

In [13]:
# check for 0db SNR
constant = calculation.get_constant(audio,noise)
power_signal = calculation.meanPower(audio)
power_noise = calculation.meanPower(constant*noise)

print(constant)

print("SNR: %s dB" %calculation.snr(power_signal,power_noise))

0.31135818
SNR: 0.0 dB
